In [1]:
!mkdir -p /root/.config/Ultralytics
!cp /kaggle/input/yolov5-font/Arial.ttf /root/.config/Ultralytics/

In [2]:
%cd /kaggle/input/norfair031py3/
!pip install commonmark-0.9.1-py2.py3-none-any.whl -f ./ --no-index
!pip install rich-9.13.0-py3-none-any.whl

!mkdir /kaggle/working/tmp
!cp -r /kaggle/input/norfair031py3/filterpy-1.4.5/filterpy-1.4.5/ /kaggle/working/tmp/
%cd /kaggle/working/tmp/filterpy-1.4.5/
!pip install .
!rm -rf /kaggle/working/tmp

# norfair
%cd /kaggle/input/norfair031py3/
!pip install norfair-0.3.1-py3-none-any.whl -f ./ --no-index

/kaggle/input/norfair031py3
Looking in links: ./
Processing ./commonmark-0.9.1-py2.py3-none-any.whl
Processing ./rich-9.13.0-py3-none-any.whl
/kaggle/working/tmp/filterpy-1.4.5
Processing /kaggle/working/tmp/filterpy-1.4.5
  Preparing metadata (setup.py) ... - done
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110474 sha256=766647f57a9d5e99d95ee4187b1d14bf992eb59d4feabe5474a092432c14df82
  Stored in directory: /root/.cache/pip/wheels/0e/ba/6d/20557ee403bc0c849d9fe3b358ded9918d7d2b0c5b0253fc99
Successfully built filterpy
/kaggle/input/norfair031py3
Looking in links: ./
Processing ./norfair-0.3.1-py3-none-any.whl


In [3]:
%cd /kaggle/working/

/kaggle/working


In [4]:
import sys
sys.path.append('../input/tensorflow-great-barrier-reef')
sys.path.append('../input/ensemble-boxes-104/ensemble_boxes-1.0.4')

import os
import cv2
import ast
import glob
import torch
import random
import shutil
import numpy as np
import pandas as pd
import albumentations as albu
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
from ensemble_boxes import *
from tqdm.notebook import tqdm
from norfair import Detection, Tracker
from scipy.spatial.distance import cdist

pd.set_option('display.max_columns', 50)

# Configurations

In [5]:
SUBMIT   = True
IMG_SIZE = 3600
CONF     = 0.50
IOU      = 0.50
IOU_WBF  = 0.50
AUGMENT  = False
FDA_aug  = False

In [6]:
def voc2yolo(bboxes, image_height=720, image_width=1280):
    """
    voc  => [x1, y1, x2, y1]
    yolo => [xmid, ymid, w, h] (normalized)
    """
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    
    bboxes[..., [0, 2]] = bboxes[..., [0, 2]]/ image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]]/ image_height
    
    w = bboxes[..., 2] - bboxes[..., 0]
    h = bboxes[..., 3] - bboxes[..., 1]
    
    bboxes[..., 0] = bboxes[..., 0] + w/2
    bboxes[..., 1] = bboxes[..., 1] + h/2
    bboxes[..., 2] = w
    bboxes[..., 3] = h
    
    return bboxes

def yolo2voc(bboxes, image_height=720, image_width=1280):
    """
    yolo => [xmid, ymid, w, h] (normalized)
    voc  => [x1, y1, x2, y1]
    """ 
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    
    bboxes[..., [0, 2]] = bboxes[..., [0, 2]]* image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]]* image_height
    
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
    bboxes[..., [2, 3]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]
    
    return bboxes

def coco2yolo(bboxes, image_height=720, image_width=1280):
    """
    coco => [xmin, ymin, w, h]
    yolo => [xmid, ymid, w, h] (normalized)
    """
    
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    
    # normolizinig
    bboxes[..., [0, 2]]= bboxes[..., [0, 2]]/ image_width
    bboxes[..., [1, 3]]= bboxes[..., [1, 3]]/ image_height
    
    # converstion (xmin, ymin) => (xmid, ymid)
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]/2
    
    return bboxes

def yolo2coco(bboxes, image_height=720, image_width=1280):
    """
    yolo => [xmid, ymid, w, h] (normalized)
    coco => [xmin, ymin, w, h]
    """ 
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    
    # denormalizing
    bboxes[..., [0, 2]]= bboxes[..., [0, 2]]* image_width
    bboxes[..., [1, 3]]= bboxes[..., [1, 3]]* image_height
    
    # converstion (xmid, ymid) => (xmin, ymin) 
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
    
    return bboxes

def voc2coco(bboxes, image_height=720, image_width=1280):
    bboxes  = voc2yolo(bboxes, image_height, image_width)
    bboxes  = yolo2coco(bboxes, image_height, image_width)
    return bboxes

def load_image(image_path):
    return cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)

In [7]:
def predict(model, img, size=768, augment=False):
    height, width = img.shape[:2]
    results = model(img, size=size, augment=augment)  # custom inference size
    preds   = results.pandas().xyxy[0]
    bboxes  = preds[['xmin','ymin','xmax','ymax']].values
    if len(bboxes):
        bboxes = voc2coco(bboxes, height, width).astype(int)
        confs  = preds.confidence.values
        return bboxes, confs
    else:
        return [],[]
    
def format_prediction(bboxes, confs):
    annot = ''
    if len(bboxes)>0:
        for idx in range(len(bboxes)):
            xmin, ymin, w, h = bboxes[idx]
            conf             = confs[idx]
            annot += f'{conf} {xmin} {ymin} {w} {h}'
            annot +=' '
        annot = annot.strip(' ')
    return annot

In [8]:
##############################################################
#                      Tracking helpers                      #
##############################################################
# Helper to convert bbox in format [x_min, y_min, x_max, y_max, score] to norfair.Detection class
def to_norfair(detects, frame_id):
    result = []
    for x_min, y_min, x_max, y_max, score in detects:
        xc, yc = (x_min + x_max) / 2, (y_min + y_max) / 2
        w, h = x_max - x_min, y_max - y_min
        result.append(Detection(points=np.array([xc, yc]), scores=np.array([score]), data=np.array([w, h, frame_id])))
        
    return result

# Euclidean distance function to match detections on this frame with tracked_objects from previous frames
def euclidean_distance(detection, tracked_object):
    return np.linalg.norm(detection.points - tracked_object.estimate)

In [9]:
def tracking_function(tracker, frame_id, bboxes, scores):
    
    detects = []
    predictions = []
    
    if len(scores)>0:
        for i in range(len(bboxes)):
            box = bboxes[i]
            score = scores[i]
            x_min = int(box[0])
            y_min = int(box[1])
            bbox_width  = int(box[2])
            bbox_height = int(box[3])
            detects.append([x_min, y_min, x_min+bbox_width, y_min+bbox_height, score])
            predictions.append('{:.2f} {} {} {} {}'.format(score, x_min, y_min, bbox_width, bbox_height))
            
    # Update tracks using detects from current frame
    tracked_objects = tracker.update(detections=to_norfair(detects, frame_id))
    for tobj in tracked_objects:
        bbox_width, bbox_height, last_detected_frame_id = tobj.last_detection.data
        if last_detected_frame_id == frame_id:  # Skip objects that were detected on current frame
            continue
        # Add objects that have no detections on current frame to predictions
        xc, yc = tobj.estimate[0]
        x_min, y_min = int(round(xc - bbox_width / 2)), int(round(yc - bbox_height / 2))
        score = tobj.last_detection.scores[0]

        predictions.append('{:.2f} {} {} {} {}'.format(score, x_min, y_min, bbox_width, bbox_height))
        
    return predictions

In [10]:
def show_image(img, boxes):
    img_pil = Image.fromarray(img.copy())
    for box in boxes:
        draw = ImageDraw.Draw(img_pil)
        x0, y0, x1, y1 = (box[0], box[1], box[0]+box[2], box[1]+box[3])
        draw.rectangle( (x0, y0, x1, y1), outline=180, width=3)
    return img_pil

def run_wbf(bboxes, confs, image_size, classs=None, iou_thr=0.50, skip_box_thr=0.0001, weights=None):
    boxes  = [bbox/(image_size-1) for bbox in bboxes]
    scores = [conf for conf in confs]    
    labels = [np.ones(conf.shape[0]) for conf in confs]
    boxes, scores, _ = weighted_boxes_fusion(boxes, scores, labels, weights=weights, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    boxes  = boxes*(image_size-1)
    return boxes, scores

# Inference

In [11]:
import greatbarrierreef
env = greatbarrierreef.make_env()# initialize the environment
iter_test = env.iter_test()      # an iterator which loops over the test set and sample submission

In [12]:
def load_model(weight_path, conf=None, iou=None):
    model = torch.hub.load('../input/yolov5-lib-ds', 
                           'custom', 
                           path=weight_path,
                           source='local',
                           force_reload=True)
    if conf is not None:
        model.conf = conf
    if iou is not None:
        model.iou  = iou
    return model

model1 = load_model('../input/reef-baseline-fold12/l6_3600_uflip_vm5_f12_up/f1/best.pt', CONF, IOU)
model2 = load_model('../input/reef-baseline-fold12/l6_3600_uflip_vm5_f12_up/f2/best.pt', CONF, IOU)
model3 = load_model('../input/yolov5-1920-4/best.pt', CONF, IOU)
model4 = load_model('../input/yolos-video-fold/yolov5s6_3000_b6_uflip_split91_f1_v12.pt', CONF, IOU)
model5 = load_model('../input/yolov5s6/f2_sub2.pt', CONF, IOU)

YOLOv5 🚀 2022-2-5 torch 1.9.1 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

Fusing layers... 
Model Summary: 280 layers, 12308200 parameters, 0 gradients
Adding AutoShape... 
YOLOv5 🚀 2022-2-5 torch 1.9.1 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

Fusing layers... 
Model Summary: 280 layers, 12308200 parameters, 0 gradients
Adding AutoShape... 
YOLOv5 🚀 2022-2-5 torch 1.9.1 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

Fusing layers... 
Model Summary: 378 layers, 35254692 parameters, 0 gradients
Adding AutoShape... 
YOLOv5 🚀 2022-2-5 torch 1.9.1 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

Fusing layers... 
Model Summary: 280 layers, 12308200 parameters, 0 gradients
Adding AutoShape... 
YOLOv5 🚀 2022-2-5 torch 1.9.1 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

Fusing layers... 
Model Summary: 280 layers, 12308200 parameters, 0 gradients
Adding AutoShape... 


In [13]:
df = pd.read_csv(f'../input/tensorflow-great-barrier-reef/train.csv')
df["image_path"] = "../input/tensorflow-great-barrier-reef/train_images/video_" + df.video_id.astype(str) + "/" + df.video_frame.astype(str) + ".jpg"
FDA_reference = df[df['annotations']!='[]']
FDA_trans     = albu.FDA(FDA_reference['image_path'].values)

In [14]:
if not SUBMIT:
    tracker = Tracker(
        distance_function=euclidean_distance, 
        distance_threshold=30,
        hit_inertia_min=3,
        hit_inertia_max=6,
        initialization_delay=1,
    )
    
    filename = [f.replace(".jpg","").zfill(5) for f in os.listdir("../input/tensorflow-great-barrier-reef/train_images/video_0/")]
    filename = sorted(filename)
    filename = [str(int(f)) + ".jpg" for f in filename]

    n = 1000
    frame_id = 0
    for idx, (f) in tqdm(enumerate(filename), total=len(filename)):
        img = cv2.imread(INPUT_PATH + f)[:, :, [2, 1, 0]]
        bboxes1, confs1  = predict(model1, img, size=IMG_SIZE, augment=AUGMENT)
        bboxes2, confs2  = predict(model2, img, size=IMG_SIZE, augment=AUGMENT)

        if 0 < len(bboxes1) and 0 < len(bboxes2):
            bboxes1[:, 2] = bboxes1[:, 2] + bboxes1[:, 0]
            bboxes1[:, 3] = bboxes1[:, 3] + bboxes1[:, 1]
            bboxes2[:, 2] = bboxes2[:, 2] + bboxes2[:, 0]
            bboxes2[:, 3] = bboxes2[:, 3] + bboxes2[:, 1]
            boxes_list    = [bboxes1, bboxes2]
            scores_list   = [confs1,  confs2]
            bboxes, confs = run_wbf(boxes_list, scores_list, IMG_SIZE, None, 0.5, 0.0001, None)
            bboxes[:, 2]  = bboxes[:, 2] - bboxes[:, 0]
            bboxes[:, 3]  = bboxes[:, 3] - bboxes[:, 1]
        else:
            bboxes = bboxes1
            confs  = confs1

        predictions    = tracking_function(tracker, frame_id, bboxes, confs)
        prediction_str = ' '.join(predictions)
        frame_id += 1

        if idx == n:
            break

In [15]:
if SUBMIT:
    tracker = Tracker(
        distance_function=euclidean_distance, 
        distance_threshold=30,
        hit_inertia_min=3,
        hit_inertia_max=6,
        initialization_delay=1,
    )

    frame_id   = 0
    for idx, (img, pred_df) in enumerate(tqdm(iter_test)):
        if FDA_aug:
            img = FDA_trans(image=img)['image']
        bboxes1, bboxes2, bboxes3, bboxes4, bboxes5 = [], [], [], [], []
        confs1,  confs2,  confs3,  confs4,  confs5  = [], [], [], [], []
        bboxes1, confs1  = predict(model1, img, size=IMG_SIZE, augment=AUGMENT)
        #bboxes2, confs2  = predict(model2, img, size=IMG_SIZE, augment=AUGMENT)
        bboxes3, confs3  = predict(model3, img, size=IMG_SIZE, augment=AUGMENT)
        #bboxes4, confs4  = predict(model4, img, size=IMG_SIZE, augment=AUGMENT)
        bboxes5, confs5  = predict(model5, img, size=IMG_SIZE, augment=AUGMENT)

        boxes_list  = []
        scores_list = []
        if 0 < len(bboxes1):
            bboxes1[:, 2] = bboxes1[:, 2] + bboxes1[:, 0]
            bboxes1[:, 3] = bboxes1[:, 3] + bboxes1[:, 1]
            boxes_list.append(bboxes1)
            scores_list.append(confs1)
        if 0 < len(bboxes2):
            bboxes2[:, 2] = bboxes2[:, 2] + bboxes2[:, 0]
            bboxes2[:, 3] = bboxes2[:, 3] + bboxes2[:, 1]
            boxes_list.append(bboxes2)
            scores_list.append(confs2)
        if 0 < len(bboxes3):
            bboxes3[:, 2] = bboxes3[:, 2] + bboxes3[:, 0]
            bboxes3[:, 3] = bboxes3[:, 3] + bboxes3[:, 1]
            boxes_list.append(bboxes3)
            scores_list.append(confs3)
        if 0 < len(bboxes4):
            bboxes4[:, 2] = bboxes4[:, 2] + bboxes4[:, 0]
            bboxes4[:, 3] = bboxes4[:, 3] + bboxes4[:, 1]
            boxes_list.append(bboxes4)
            scores_list.append(confs4)
        if 0 < len(bboxes5):
            bboxes5[:, 2] = bboxes5[:, 2] + bboxes5[:, 0]
            bboxes5[:, 3] = bboxes5[:, 3] + bboxes5[:, 1]
            boxes_list.append(bboxes5)
            scores_list.append(confs5)

        if 0 < len(boxes_list):
            bboxes, confs = run_wbf(boxes_list, scores_list, IMG_SIZE, None, IOU_WBF, 0.0001, None)
            bboxes[:, 2]  = bboxes[:, 2] - bboxes[:, 0]
            bboxes[:, 3]  = bboxes[:, 3] - bboxes[:, 1]
        else:
            bboxes = bboxes1
            confs  = confs1

        predictions    = tracking_function(tracker, frame_id, bboxes, confs)
        prediction_str = ' '.join(predictions)
        pred_df['annotations'] = prediction_str
        env.predict(pred_df)
        frame_id += 1

0it [00:00, ?it/s]

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.


In [16]:
sub_df = pd.read_csv('submission.csv')
sub_df.head()

,index,annotations
0,0,NaN
1,1,NaN
2,2,NaN
